# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,east london,-33.0153,27.9116,67.53,32,58,5.61,ZA,1723059588
1,1,puerto ayora,-0.7393,-90.3518,76.96,89,30,7.00,EC,1723059471
2,2,waitangi,-43.9535,-176.5597,49.82,69,18,15.59,NZ,1723059590
3,3,twentynine palms,34.1356,-116.0542,98.74,34,53,4.72,US,1723059591
4,4,sinabang,2.4803,96.3801,79.50,82,100,2.86,ID,1723059592


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [17]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    alpha = 0.5
)

# Display the map
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [21]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[(city_data_df["Max Temp"] < 75) & (city_data_df["Max Temp"] > 55) & (city_data_df["Wind Speed"] < 4) & (city_data_df["Cloudiness"] < 5)]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
104,104,tromso,69.6496,18.9570,68.02,52,0,2.30,NO,1723059494
151,151,lata,40.1629,-8.3327,62.40,82,0,3.67,PT,1723059766
235,235,viitasaari,63.0667,25.8667,59.09,71,0,3.74,FI,1723059864
250,250,port elizabeth,-33.9180,25.5701,58.32,57,0,2.30,ZA,1723059881
315,315,okhotsk,59.3833,143.3000,55.26,90,0,3.33,RU,1723059960
331,331,hwange,-18.3693,26.5019,69.03,39,0,3.06,ZW,1723059980
341,341,pechory,57.8164,27.6119,62.06,87,0,3.94,RU,1723059991
403,403,skrunda,56.6833,22.0167,60.57,93,1,3.78,LV,1723060067
428,428,ulan-ude,51.8261,107.6098,55.29,94,2,2.24,RU,1723060096
487,487,bima,-8.4667,118.7167,72.84,97,4,3.18,ID,1723060168


### Step 3: Create a new DataFrame called `hotel_df`.

In [22]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
104,tromso,NO,69.6496,18.9570,52,
151,lata,PT,40.1629,-8.3327,82,
235,viitasaari,FI,63.0667,25.8667,71,
250,port elizabeth,ZA,-33.9180,25.5701,57,
315,okhotsk,RU,59.3833,143.3000,90,
331,hwange,ZW,-18.3693,26.5019,39,
341,pechory,RU,57.8164,27.6119,87,
403,skrunda,LV,56.6833,22.0167,93,
428,ulan-ude,RU,51.8261,107.6098,94,
487,bima,ID,-8.4667,118.7167,97,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [23]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tromso - nearest hotel: Clarion Collection Hotel Aurora
lata - nearest hotel: Residencial Botânico
viitasaari - nearest hotel: No hotel found
port elizabeth - nearest hotel: Waterford Hotel
okhotsk - nearest hotel: No hotel found
hwange - nearest hotel: No hotel found
pechory - nearest hotel: Печоры Парк
skrunda - nearest hotel: No hotel found
ulan-ude - nearest hotel: Хуторок
bima - nearest hotel: No hotel found
chifeng - nearest hotel: 心连心酒店
barcelos - nearest hotel: Hotel Dom Nuno


,City,Country,Lat,Lng,Humidity,Hotel Name
104,tromso,NO,69.6496,18.9570,52,Clarion Collection Hotel Aurora
151,lata,PT,40.1629,-8.3327,82,Residencial Botânico
235,viitasaari,FI,63.0667,25.8667,71,No hotel found
250,port elizabeth,ZA,-33.9180,25.5701,57,Waterford Hotel
315,okhotsk,RU,59.3833,143.3000,90,No hotel found
331,hwange,ZW,-18.3693,26.5019,39,No hotel found
341,pechory,RU,57.8164,27.6119,87,Печоры Парк
403,skrunda,LV,56.6833,22.0167,93,No hotel found
428,ulan-ude,RU,51.8261,107.6098,94,Хуторок
487,bima,ID,-8.4667,118.7167,97,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [24]:
%%capture --no-display

# Configure the map plot
humidity_map = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    alpha = 0.5,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)